<a href="https://colab.research.google.com/github/ez0ez0/ko-Llama-3.1-8B/blob/main/Llama_3_1_8B_%EB%AF%B8%EC%84%B8_%EC%A1%B0%EC%A0%95_(LoRA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-3.1-8B 미세 조정 (LoRA)
- [출처](https://gautam75.medium.com/fine-tuning-llama-3-1-8b-for-function-calling-using-lora-159b9ee66060)

In [1]:
!pip install unsloth "xformers==0.0.28.post2"
!pip install wandb -qU

### 1. 가중치 및 바이어스 설정
- 모델의 미세 조정 프로세스 모니터링, 기록 --> W&B(wandb)를 구성

In [2]:
# api키 등록
from google.colab import userdata
import os

WB_API_KEY = userdata.get('WB_API_KEY')
os.environ["WB_API_KEY"] = WB_API_KEY

HF_TOKEN=userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = HF_TOKEN


In [3]:
import wandb
# from dotenv import load_dotenv
# load_dotenv()

def  setup_wandb ( project_name: str , run_name: str ):
    # API 키 설정
    try :
        wb_api_key = os.getenv( "WB_API_KEY" )
        wandb.login(key=wb_api_key)
        print ( "WandB에 성공적으로 로그인했습니다." )
    except KeyError:
        raise EnvironmentError( "WANDB_API_KEY가 환경 변수에 설정되지 않았습니다." )
    except Exception as e:
        print ( f"WandB에 로그인하는 중 오류 발생: {e} " )

    # 선택 사항: 모델 기록
    os.environ[ "WANDB_LOG_MODEL" ] = "checkpoint"
    os.environ[ "WANDB_WATCH" ] = "all"
    os.environ[ "WANDB_SILENT" ] = "true"

    # WandB 실행 초기화
    try :
        wandb.init(project=project_name, name=run_name)
        print ( f"WandB 실행이 초기화되었습니다: 프로젝트 - {project_name} , 실행 - {run_name} " )
    except Exception as e:
        print ( f"WandB 실행을 초기화하는 동안 오류가 발생했습니다: {e} " )


setup_wandb(project_name= "<project_name>" , run_name= "<run_name>" )

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


WandB에 성공적으로 로그인했습니다.


wandb: Currently logged in as: dabidiboo90 (dabidiboo90-puzzle-systems). Use `wandb login --relogin` to force relogin


WandB 실행이 초기화되었습니다: 프로젝트 - <project_name> , 실행 - <run_name> 


### 2. 허깅페이스 인증

In [4]:
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

if hf_token is None:
    raise EnvironmentError("HF_TOKEN is not set in the environment variables.")
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### 3. base model, tokenizer 로딩
- model : unsloth/Meta-Llama-3.1-8B-Instruct
- 구성 가능한 시퀀스 길이 (max_seq_length=2048)는 모델이 처리할 수 있는 최대 입력 시퀀스 길이를 지정하며, 이는 종종 모델의 컨텍스트 길이라고 합니다.
- 유연한 최적화를 위한 자동 dtype 감지 (dtype=None)
- 4비트 양자화 옵션 (load_in_4bit=False)을 사용하면 더 높은 모델 충실도를 위해 기본 정밀도를 사용할 수 있습니다.

In [5]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 2048     # Unsloth는 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
dtype = None              # 자동 감지 없음
load_in_4bit = False      # 메모리 사용량을 줄이기 위해 4비트 양자화를 사용합니다. False가 될 수 있습니다.

# Unsloth의 를 사용하여 모델과 토크나이저를 로드합니다 :: FastLanguageModel.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

### 4. 효율적인 미세 조정을 위한 LoRA 구성
- LoRA를 사용하여 매개변수 효율적 미세 조정(PEFT)을 위한 Llama-3.1 모델을 구성하여 **메모리 사용을 최소화** & 전체 모델 대신 특정 구성 요소만 조정하여 **학습을 가속화**하여, 선택한 레이어의 효율적인 미세 조정을 가능하게 합니다.

- #### 파라미터 세부 내용
  - r=16: LoRA 행렬의 순위를 설정하고 모델 성능과 메모리 사용량의 균형을 맞춥니다.
  - target_modules: 특정 미세 조정을 위해 "q_proj"및 와 같은 레이어를 식별합니다."k_proj"
  - lora_alpha=16: 과도한 맞춤을 방지하기 위해 스케일링 요소를 제어합니다.
  - lora_dropout=0: 일관된 학습을 위해 드롭아웃을 0으로 설정합니다.
  - use_gradient_checkpointing="unsloth": 특히 컨텍스트 길이가 긴 경우 메모리 사용량을 최소화합니다.
  - bias="none”: 추가 바이어스 항목을 생략합니다.
  - random_state=3407: 재현 가능한 훈련이 실행되도록 합니다.
  - use_rslora=False:표준적이고 덜 복잡한 작업을 최적화하는 rank-sensitive LoRA를 비활성화함.
  - loftq_config=None: LoftQ를 비활성화합니다. LoftQ는 정확도를 높이기 위해 고급 초기화를 사용하지만 처음에 메모리 사용량이 더 많아집니다.


In [10]:
# 모델 성능을 유지하면서도 리소스 효율적인 미세 조정이 가능한 설정
model = FastLanguageModel.get_peft_model(
    model,
    r= 16 ,    # LoRA 순위 - 제안 값: 8, 16, 32, 64, 128
    target_modules=[ "q_proj" , "k_proj" , "v_proj" , "o_proj" ,
                    "gate_proj" , "up_proj" , "down_proj" ],
    lora_alpha= 16 ,
    lora_dropout= 0 ,    # 모두 지원하지만 = 0으로 최적화됨
    bias= "none" ,       # 모두 지원하지만 = "none"으로 최적화됨
    use_gradient_checkpointing= "unsloth" ,   # 장기 컨텍스트 튜닝에 이상적
    random_state= 3407 ,
    use_rslora= False ,    # 더 간단한 작업을 위해 순위에 민감한 LoRA 비활성화
    loftq_config= None    # 표준 미세 튜닝의 경우 LoftQ 사용 안 함
 )

Unsloth: Already have LoRA adapters! We shall skip this step.


### 5. 데이터 세트 로딩 및 처리
- 미세 조정을 시작하기 위해 전체 데이터 세트를 사용하는 대신 데이터 세트에서 15,000개의 관리 가능한 하위 세트로 시작하겠습니다. 이를 통해 모델의 성능을 일찍 평가하고 보다 효율적으로 조정할 수 있습니다. 10~20,000개의 샘플 크기는 적절한 균형을 이룹니다. 메모리와 교육 시간 요구 사항을 합리적으로 유지하면서도 의미 있는 통찰력을 얻을 수 있을 만큼 충분히 큽니다.

In [11]:
from datasets import load_dataset

## 수정한 코드
# 데이터세트 로딩
dataset = load_dataset("CarrotAI/ko-instruction-dataset", split="train", token=hf_token)
print ( f" 미세 조정을 위해 {len(dataset)}의 샘플 크기 사용." )

## 원래 코드
# dataset = load_dataset("Salesforce/xlam-function-calling-60k", split="train", token=hf_token)
# 미세 조정을 위해 15,000개의 샘플 하위 집합 선택
# dataset = dataset.select(range(15000))
# print ( f" 미세 조정을 위해 {len(dataset)}의 샘플 크기 사용." )

 미세 조정을 위해 7040의 샘플 크기 사용.


### 6. 데이터를 모델과 호환되는 토큰으로 변환
- Unsloth의 채팅 템플릿을 사용하여 원시 데이터를 모델과 호환되는 토큰으로 변환합니다. 이 단계는 프롬프트를 표준화하여 모델이 구조화된 방식으로 출력을 이해하고 예측할 수 있도록 합니다.

In [12]:
from unsloth.chat_templates import get_chat_template

# 채팅 템플릿과 매핑을 사용하여 토크나이저 초기화
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT 스타일
    map_eos_token = True,        # 대신 <|im_end|>를 <|eot_id|>에 매핑
)

def formatting_prompts_func(examples):
    convos = []

    # 배치의 각 항목을 반복합니다(예제는 값 목록으로 구성됨)

    ## 수정한 코드
    for query, answers in zip(examples['instruction'], examples['output']):
        tool_user = {
            "content": f"당신은 인공지능 어시스턴트입니다. 친절하고 상세한 답변을 해주세요.",
            "role": "system"
        }
        ques_user = {
            "content": f"{query}",
            "role": "user"
        }
        assistant = {
            "content": f"{answers}",
            "role": "assistant"
        }
        convos.append([tool_user, ques_user, assistant])

    ## 원래 코드
    # for query, tools, answers in zip(examples['query'], examples['tools'], examples['answers']):
    #     tool_user = {
    #         "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{tools}",
    #         "role": "system"
    #     }
    #     ques_user = {
    #         "content": f"{query}",
    #         "role": "user"
    #     }
    #     assistant = {
    #         "content": f"{answers}",
    #         "role": "assistant"
    #     }
    #     convos.append([tool_user, ques_user, assistant])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# 데이터세트에 서식 적용
dataset = dataset.map(formatting_prompts_func, batched = True,)


#### 7. TrainingArguments 정의
- 이 설정은 모델을 미세 조정하기 위한 하이퍼파라미터와 로깅 구성을 정의하여 잘 제어된 단계로 효율적인 학습을 유지하는 데 도움을 줌. 각 매개변수는 모델 동작을 최적화하고 진행 상황을 효과적으로 모니터링하는데 역할을 합니다.

In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
        per_device_train_batch_size = 8,  # device 당 batch size를 제어
        gradient_accumulation_steps = 2,  # 더 큰 batch배치를 시뮬레이션하기 위해 그래디언트를 축적
        warmup_steps = 5,
        learning_rate = 2e-4,             # 최적화를 위한 학습 속도를 설정
        num_train_epochs = 3,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,              # 과적합을 방지하기 위한 정규화
        lr_scheduler_type = "linear",     # Chooses a linear learning rate decay
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",              # Weights & Biases (W&B) logging 활성화
        logging_steps = 1,                # W&B 로깅 빈도를 설정
        logging_strategy = "steps",       # 지정된 각 단계에서 메트릭을 로깅합니다.
        save_strategy = "no",
        load_best_model_at_end = True,    # 마지막에 최상의 모델을 로드
        save_only_model = False           # 가중치뿐만 아니라 모델 전체를 저장
    )

### 8. SFTTrainer와 Unsloth를 이용한 훈련
- SFTTrainer는 사용자 정의 토큰화, 데이터 세트 전처리 및 메모리 최적화를 통한 감독된 미세 조정을 위해 구성됩니다.
- unsloth_train 와 SFTTrainer 가 조합하면, 긴 시퀀스를 처리하고 메모리 사용량을 줄이는데 중요한 unsloth's optimized gradient checkpointing 이 가능합니다.

In [14]:
!pip install trl

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # 짧은 시퀀스의 경우 학습 속도를 5배 더 빠르게 할 수 있습니다.
    args = args
)

Map (num_proc=2):   0%|          | 0/7040 [00:00<?, ? examples/s]

###  훈련 시작 시 초기 GPU 메모리 통계를 캡처

In [16]:
# 현재 메모리 통계 표시
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
15.648 GB of memory reserved.


### 9. 모델학습

In [ ]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,040 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 1,320
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,1.341300
2,1.217100
3,1.255300
4,1.238800
5,1.165000
6,1.151800
7,1.124000
8,1.160300
9,0.943700
10,0.975900


####training 후 최종 메모리 사용 확인

In [ ]:
wandb.finish()

In [ ]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

### 10. 모델 저장

In [ ]:
lora_model_name = "ko-Meta-Llama-3.1-8B-Instruct-LoRA"
model_name = "ko-Meta-Llama-3.1-8B-Instruct"
hf_username = "Leejy0-0"

#### LoRA Adapter 저장

In [ ]:
# Local saving
model.save_pretrained(lora_model_name)
tokenizer.save_pretrained(lora_model_name)

# Online saving
model.push_to_hub(f"{hf_username}/{lora_model_name}", token = hf_token)
tokenizer.push_to_hub(f"{hf_username}/{lora_model_name}", token = hf_token)

#### 모델(base model + LoRA Adapter)을 16bit 정밀도로 저장(vLLM)
- vLLM : 대규모 언어 모델(LLM)을 더 빠르고 효율적으로 서빙하기 위한 오픈소스 시스템입니다.특히 모델의 응답 속도와 메모리 효율성을 높이기 위해 최적화되었으며, 대규모 언어 모델의 실시간 애플리케이션을 지원하는 데 중점을 둡니다.

In [ ]:
# Merge to 16bit
model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged((f"{hf_username}/{model_name}", tokenizer, save_method = "merged_16bit", token = hf_token)

### 11. Fine-Tuned Model Evaluation


In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

max_seq_length = 2048
dtype = None
load_in_4bit = False
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"{hf_username}/{model_name}",        # 로컬 또는 hugingface에서 학습된 모델 불러오기
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)  # 네이티브 2배 빠른 추론을 활성화

### 12. 모델 실행

In [ ]:
query = "헤지 추측의 중요성과 현재의 연구 상황을 고려하여, 이 추측이 해결되었을 때 수학 분야에 미칠 수 있는 영향에 대해 논의해 보세요. 또한, 헤지 추측과 관련된 다른 중요한 수학적 문제들을 언급하고, 이러한 문제들이 서로 어떻게 연결되어 있는지에 대해서도 설명해 주세요."

chat = [
    {"role":"system","content": "당신은 인공지능 어시스턴트입니다. 친절하고 상세한 답변을 해주세요."},
    {"role": "user", "content": query }
]

In [ ]:
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
response = tokenizer.batch_decode(outputs)[0]
print(response)